In [2]:
import cv2 
img = cv2.imread('./Images/glassper1.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [9]:
sift = cv2.SIFT_create()
keypoints, descriptors = sift.detectAndCompute(img, None)
sift_image = cv2.drawKeypoints(gray, keypoints, img)
cv2.imshow('image', sift_image)
cv2.imwrite("./Images/glassper1-sift.jpg", sift_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
img1 = cv2.imread('./Images/glassper2.jpg')  
img2 = cv2.imread('./Images/glassper1.jpg')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
sift = cv2.SIFT_create()
keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)
matches = bf.match(descriptors_1,descriptors_2)
matches = sorted(matches, key = lambda x:x.distance)
matched_img = cv2.drawMatches(img1, keypoints_1, img2, keypoints_2, matches[:50], img2, flags=2)
cv2.imshow('image', matched_img)
cv2.imwrite("./Images/matchedper.jpg", matched_img)
cv2.waitKey(0)
cv2.destroyAllWindows()